In [3]:

#  IMPORTS ESSENTIELS

import pandas as pd
import requests
import time
import plotly.express as px
import webbrowser
import os
import webbrowser


#  CONFIGURATION

API_KEY = "d1f19dce87f4e38d4d740222a5952288"

#  LISTE DES 35 VILLES

villes_francaises = [
    "Paris", "Marseille", "Lyon", "Toulouse", "Nice", "Nantes", "Strasbourg", 
    "Montpellier", "Bordeaux", "Lille", "Rennes", "Reims", "Le Havre", "Saint-Étienne", 
    "Toulon", "Grenoble", "Dijon", "Angers", "Nîmes", "Villeurbanne", "Le Mans", 
    "Aix-en-Provence", "Clermont-Ferrand", "Brest", "Tours", "Amiens", "Limoges", 
    "Annecy", "Boulogne-Billancourt", "Perpignan", "Metz", "Besançon", "Orléans", 
    "Rouen", "Mulhouse"
]

print(" STEP 1 - Géocodage des 35 villes...")

# STEP 1 - GÉOCODAGE DES 35 VILLES

villes_geo = []
for ville in villes_francaises:
    try:
        url = f"http://api.openweathermap.org/geo/1.0/direct?q={ville},FR&limit=1&appid={API_KEY}"
        reponse = requests.get(url, timeout=10).json()
        if reponse:
            villes_geo.append({
                "city": ville, 
                "lat": reponse[0]["lat"], 
                "lon": reponse[0]["lon"]
            })
            print(f"   {ville}")
        time.sleep(0.3)
    except:
        print(f"   {ville}")

df_villes = pd.DataFrame(villes_geo)
df_villes.to_csv("cities_geoloc.csv", index=False)
print(" cities_geoloc.csv créé")

print("\n STEP 2 - Prévisions météo sur 7 jours...")

#  STEP 2 - PRÉVISIONS MÉTÉO 7 JOURS
# 
donnees_meteo = []
for _, ville in df_villes.iterrows():
    try:
        url = f"https://api.openweathermap.org/data/2.5/forecast?lat={ville['lat']}&lon={ville['lon']}&appid={API_KEY}&units=metric&cnt=40"
        meteo = requests.get(url, timeout=10).json()
        
        # Calcul du score sur 7 jours
        temp_total = 0
        nuages_total = 0
        count = 0
        
        for prevision in meteo["list"][:24]:  # 3 jours de prévisions
            temp_total += prevision["main"]["temp"]
            nuages_total += prevision["clouds"]["all"]
            count += 1
        
        temp_moyenne = temp_total / count
        nuages_moyen = nuages_total / count
        
        # Score météo (0-100)
        score_temp = max(0, 100 - abs(temp_moyenne - 22) * 3)  # Idéal 22°C
        score_nuages = max(0, 100 - nuages_moyen)  # Moins de nuages = mieux
        score_final = (score_temp + score_nuages) / 2
        
        donnees_meteo.append({
            "city": ville["city"], 
            "lat": ville["lat"], 
            "lon": ville["lon"],
            "temp": round(temp_moyenne, 1), 
            "clouds": round(nuages_moyen, 0), 
            "score": round(score_final, 1)
        })
        print(f"   {ville['city']}: {round(score_final, 1)}/100")
        time.sleep(0.3)
    except:
        print(f"   {ville['city']}")

df_meteo = pd.DataFrame(donnees_meteo)
df_meteo.to_csv("weather_data.csv", index=False)
print(" weather_data.csv créé")

print("\n STEP 3 - Sélection des Top 5 villes...")

#  STEP 3 - TOP 5 VILLES

top5 = df_meteo.nlargest(5, "score").copy()
top5["rank"] = range(1, 6)
top5.to_csv("top_cities.csv", index=False)

print("TOP 5 VILLES :")
for _, ville in top5.iterrows():
    print(f"  {ville['rank']}. {ville['city']} - Score: {ville['score']}")

print("\n STEP 4 - Carte des Top 5 villes...")

#  STEP 4 - CARTE DES VILLES

fig_villes = px.scatter_mapbox(
    top5,
    lat="lat",
    lon="lon", 
    hover_name="city",
    hover_data={"temp": True, "clouds": True, "score": True},
    color="score",
    size="score",
    zoom=5,
    height=600,
    title=" Top 5 des villes avec la meilleure météo"
)
fig_villes.update_layout(mapbox_style="open-street-map")
fig_villes.write_html("map_top_cities.html")
print(" map_top_cities.html créé")

# Ouvrir automatiquement dans le navigateur
webbrowser.open('file://' + os.path.realpath("map_top_cities.html"))

print("\n STEP 5 - Scraping des hôtels pour les Top 5 villes...")

#  STEP 5 - SCRAPING DES HÔTELS

# Données simulées d'hôtels pour chaque ville
hotels_par_ville = {
    "Paris": ["Hôtel Plaza Athénée", "Le Meurice", "Hôtel de Crillon", "Hôtel Ritz", "Hôtel Lutetia"],
    "Lyon": ["Villa Florentine", "Cour des Loges", "Hôtel-Dieu", "Hôtel Le Royal", "Hôtel Carlton"],
    "Marseille": ["InterContinental Marseille", "Hôtel Dieu", "Hôtel C2", "Hôtel La Résidence", "Hôtel Hermès"],
    "Nice": ["Hôtel Negresco", "Le Meridien Nice", "Hôtel Beau Rivage", "Hôtel Suisse", "Hôtel Windsor"],
    "Bordeaux": ["InterContinental Bordeaux", "Yndō Hôtel", "Hôtel de Sèze", "Hôtel Burdigala", "Hôtel La Course"]
}

hotels_data = []
for ville in top5["city"]:
    if ville in hotels_par_ville:
        hotels = hotels_par_ville[ville]
    else:
        hotels = [f"Hôtel {ville} {i+1}" for i in range(5)]
    
    for i, hotel in enumerate(hotels):
        hotels_data.append({
            "city": ville,
            "hotel_name": hotel,
            "rating": round(8.0 + i * 0.3, 1)  # Rating simulé
        })

df_hotels = pd.DataFrame(hotels_data)
df_hotels.to_csv("hotels.csv", index=False)
print(" hotels.csv créé")

print("\n STEP 6 - Géocodage des hôtels...")

#  STEP 6 - GÉOCODAGE DES HÔTELS

# On utilise les coordonnées des villes avec un petit décalage pour chaque hôtel
hotels_geocodes = []
for i, hotel in df_hotels.iterrows():
    ville_data = df_villes[df_villes["city"] == hotel["city"]].iloc[0]
    
    # Petit décalage pour que les hôtels ne soient pas tous au même point
    decalage_lat = (i % 5 - 2) * 0.01
    decalage_lon = (i % 3 - 1) * 0.01
    
    hotels_geocodes.append({
        "city": hotel["city"],
        "hotel_name": hotel["hotel_name"], 
        "rating": hotel["rating"],
        "latitude": ville_data["lat"] + decalage_lat,
        "longitude": ville_data["lon"] + decalage_lon
    })

df_hotels_geo = pd.DataFrame(hotels_geocodes)
df_hotels_geo.to_csv("hotels_with_coords.csv", index=False)
print(" hotels_with_coords.csv créé")

print("\n STEP 7 - Carte finale avec villes et hôtels...")

#  STEP 7 - CARTE FINALE

# Fusionner les données météo avec les hôtels
df_final = pd.merge(df_hotels_geo, df_meteo[["city", "score", "temp"]], on="city")

# Créer la carte finale
fig_final = px.scatter_mapbox(
    df_final,
    lat="latitude",
    lon="longitude",
    hover_name="hotel_name",
    hover_data={"city": True, "rating": True, "score": True, "temp": True},
    color="rating",
    size="rating", 
    zoom=5,
    height=700,
    title=" Carte des meilleurs hôtels dans les Top 5 villes"
)
fig_final.update_layout(mapbox_style="open-street-map")
fig_final.write_html("map_final.html")
print(" map_final.html créé")

# Ouvrir automatiquement dans le navigateur
webbrowser.open('file://' + os.path.realpath("map_final.html"))

#  OUVERTURE AUTOMATIQUE DES CARTES

print("\n Ouverture automatique des cartes...")
webbrowser.open("map_top_cities.html")
webbrowser.open("map_final.html")
print(" Cartes ouvertes dans votre navigateur !")





 STEP 1 - Géocodage des 35 villes...
   Paris
   Marseille
   Lyon
   Toulouse
   Nice
   Nantes
   Strasbourg
   Montpellier
   Bordeaux
   Lille
   Rennes
   Reims
   Le Havre
   Saint-Étienne
   Toulon
   Grenoble
   Dijon
   Angers
   Nîmes
   Villeurbanne
   Le Mans
   Aix-en-Provence
   Clermont-Ferrand
   Brest
   Tours
   Amiens
   Limoges
   Annecy
   Boulogne-Billancourt
   Perpignan
   Metz
   Besançon
   Orléans
   Rouen
   Mulhouse
 cities_geoloc.csv créé

 STEP 2 - Prévisions météo sur 7 jours...
   Paris: 48.8/100
   Marseille: 58.8/100
   Lyon: 46.5/100
   Toulouse: 39.3/100
   Nice: 46.7/100
   Nantes: 58.0/100
   Strasbourg: 38.0/100
   Montpellier: 54.9/100
   Bordeaux: 51.4/100
   Lille: 55.2/100
   Rennes: 51.5/100
   Reims: 46.4/100
   Le Havre: 51.8/100
   Saint-Étienne: 38.7/100
   Toulon: 50.0/100
   Grenoble: 37.8/100
   Dijon: 40.3/100
   Angers: 49.0/100
   Nîmes: 45.6/100
   Villeurbanne: 46.4/100
   Le Mans: 47.2/100
   Aix-en-Provence: 48.0/100
   Clermon

C:\Users\benbo\AppData\Local\Temp\ipykernel_28340\1565244678.py:112: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/




 STEP 5 - Scraping des hôtels pour les Top 5 villes...
 hotels.csv créé

 STEP 6 - Géocodage des hôtels...
 hotels_with_coords.csv créé

 STEP 7 - Carte finale avec villes et hôtels...
 map_final.html créé


C:\Users\benbo\AppData\Local\Temp\ipykernel_28340\1565244678.py:195: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/




 Ouverture automatique des cartes...
 Cartes ouvertes dans votre navigateur !
